In [69]:
import ete3
import re
import os
import subprocess

%cd ~/Downloads/haitao/

/Users/thiberio/Downloads/haitao


In [55]:
species_tree = ete3.Tree(
    'Species Tree/2019_05_23_2nd_Version_Aligned_Full_Name_All_Ribosomal_Proteins.fasta.treefile',
    format=1
)
gene_tree = ete3.Tree(
    'Gene Tree/2019_05_23_Aligned_BVMonooxygenase_Species-on-SpeciesTree.fasta.treefile',
    format=1
)

In [56]:
genomes = set()
for leaf in species_tree.get_leaves():
    leaf.name = re.sub('[._-]', '', leaf.name)
    if leaf.name.startswith('SAR202'):
        genomes.add(leaf.name)
species_tree.prune(genomes)

In [57]:
print(species_tree)


                  /-SAR202clusterbacteriumIo17ChloroG9
               /-|
            /-|   \-SAR202clusterbacteriumIo17ChloroG2
           |  |
         /-|   \-SAR202clusterbacteriumMPSIndSRR3963457G2
        |  |
        |  |   /-SAR202clusterbacteriumIo17ChloroG4
        |   \-|
        |      \-SAR202clusterbacteriumIo17ChloroG7
        |
      /-|      /-SAR202clusterbacteriumCaspChloroG2
     |  |     |
     |  |     |      /-SAR202clusterbacteriumIo17ChloroG8
     |  |   /-|     |
     |  |  |  |   /-|      /-SAR202clusterbacteriumMPNPacSRR3961935G1
     |  |  |  |  |  |   /-|
     |  |  |  |  |   \-|   \-SAR202clusterbacteriumMPSIndSRR3963457G1
   /-|   \-|   \-|     |
  |  |     |     |      \-SAR202clusterbacteriumMPSAtlSRR3965592G1
  |  |     |     |
  |  |     |      \-SAR202clusterbacteriumIo17ChloroG5
  |  |     |
  |  |      \-SAR202clusterbacteriumIo17ChloroG6
--|  |
  |   \-SAR202clusterbacteriumCaspChloroG3
  |
  |      /-SAR202clusterbacteriumIo17ChloroG3
  |   /-|

In [58]:
gene_taxa = set()
for leaf in gene_tree.get_leaves():
    species = re.search('__(\S+)$', leaf.name.strip('_'), re.M).group(1)
    species = re.sub('[._-]', '', species)
    gene_id = re.search('^([^_]+)', leaf.name.strip('_'), re.M).group(1)
    gene_id = re.sub('\.\d+$', '', gene_id, re.M)
    leaf.name = '%s_%s' % (species, gene_id)
    if species.startswith('SAR202'):
        gene_taxa.add(leaf.name)
gene_tree.prune(gene_taxa)

In [64]:
# outgroup as predicted by MAD
outgroup_taxa = ['SAR202clusterbacteriumCaspChloroG4_PKB60949',
                 'SAR202clusterbacteriumIo17ChloroG3_PKB65232', 
                 'SAR202clusterbacteriumCaspChloroG4_PKB60949']
outgroup = gene_tree.get_common_ancestor(outgroup_taxa)
gene_tree.set_outgroup(outgroup)

In [65]:
print(gene_tree)


      /-SAR202clusterbacteriumCaspChloroG4_PKB60949
   /-|
  |  |   /-SAR202clusterbacteriumIo17ChloroG3_PKB65232
  |   \-|
  |      \-SAR202clusterbacteriumIo17ChloroG1_PKB63094
  |
  |               /-SAR202clusterbacteriumIo17ChloroG6_PKB71851
  |              |
  |            /-|      /-SAR202clusterbacteriumCaspChloroG2_PKB58950
  |           |  |   /-|
  |           |   \-|   \-SAR202clusterbacteriumMPSIndSRR3963457G1_PKB80840
  |         /-|     |
  |        |  |      \-SAR202clusterbacteriumIo17ChloroG8_PKB74429
  |        |  |
--|        |  |   /-SAR202clusterbacteriumIo17ChloroG9_PKB82162
  |      /-|   \-|
  |     |  |     |   /-SAR202clusterbacteriumIo17ChloroG9_PKB82764
  |     |  |      \-|
  |     |  |         \-SAR202clusterbacteriumIo17ChloroG2_PKB63926
  |     |  |
  |   /-|   \-SAR202clusterbacteriumAe2ChloroG2_PKB60442
  |  |  |
  |  |  |   /-SAR202clusterbacteriumIo17ChloroG7_PKB73514
  |  |  |  |
  |  |  |  |         /-SAR202clusterbacteriumIo17ChloroG5_PKB70198


In [66]:
species_tree.write(outfile='species.tre', format=5, dist_formatter='%.20f')
gene_tree.write(   outfile='gene.tre',    format=5, dist_formatter='%.20f')

In [75]:
ranger_input = open('ranger.input', 'w')
ranger_input.write('%s\n%s' % (
    species_tree.write(format=5, dist_formatter='%.20f'),
    gene_tree.write(   format=5, dist_formatter='%.20f'))
                  )
ranger_input.close()

In [89]:
#os.mkdir('reconciliations/')
for loop in range(500):
    subprocess.call(['/work/ranger/CorePrograms/Ranger-DTL.mac',
                     '-i',
                     'ranger.input',
                     '-o',
                     'reconciliations/ranger.output.%i' % (loop+1), 
                     '--seed',
                     str(loop+1)
                    ])
aggregate = subprocess.getoutput('/work/ranger/CorePrograms/AggregateRanger.mac reconciliations/ranger.output.')
out = open('aggregated.ranger', 'w')
out.write(aggregate)
out.close()

In [82]:
subprocess.call?